# 利用朴素贝叶斯进行新闻文本分类
初探文本分类，本文使用的数据是5000条中文新闻文本数据，目的是使用朴素贝叶斯算法，对中文新闻文本进行分类预测。流程如下：

![](http://aliyuntianchipublic.cn-hangzhou.oss-pub.aliyun-inc.com/public/files/image/null/1537169751381_w3Q9M0xu8G.jpg)

### 文本数据载入及清洗
搜狗新闻数据源：http://www.sogou.com/labs/resource/ca.php

我们从搜狗下载的数据是类似XML的带标签对的数据，因此需要使用正则表达式或者BeautifulSoup等工具处理为dataframe格式，如下图，大家通过网络爬虫获取的数据处理方法也类似。

![](http://aliyuntianchipublic.cn-hangzhou.oss-pub.aliyun-inc.com/public/files/image/null/1537169789508_9zbmmGe2sX.jpg)

文本数据提取这里就不赘述了，下面直接读入处理后的csv文件

In [2]:
import pandas as pd
import jieba
data = pd.read_table('./dataset/news_data.txt')
data.head(10)

F:\anaconda\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: read_table is deprecated, use read_csv instead, passing sep='\t'.
  This is separate from the ipykernel package so we can avoid doing imports until


,类别,新闻标题,新闻内容
0,汽车,新辉腾 ４．２ Ｖ８ ４座加长Ｉｎｄｉｖｉｄｕａｌ版２０１１款 最新报价,经销商 电话 试驾／订车Ｕ憬杭州滨江区江陵路１７８０号４００８－１１２２３３转５８６４＃保常...
1,汽车,９１８ Ｓｐｙｄｅｒ概念车,呼叫热线 ４００８－１００－３００ 服务邮箱 ｋｆ＠ｐｅｏｐｌｅｄａｉｌｙ．ｃｏｍ．ｃｎ
2,汽车,日内瓦亮相 ＭＩＮＩ性能版／概念车－１．６Ｔ引擎,ＭＩＮＩ品牌在二月曾经公布了最新的ＭＩＮＩ新概念车Ｃｌｕｂｖａｎ效果图，不过现在在日内瓦车展...
3,汽车,清仓大甩卖一汽夏利Ｎ５威志Ｖ２低至３．３９万,清仓大甩卖！一汽夏利Ｎ５、威志Ｖ２低至３．３９万＝日，启新中国一汽强势推出一汽夏利Ｎ５、威志...
4,汽车,大众敞篷家族新成员 高尔夫敞篷版实拍,在今年３月的日内瓦车展上，我们见到了高尔夫家族的新成员，高尔夫敞篷版，这款全新敞篷车受到了众...
5,汽车,悦翔两厢 １．５ ＡＴ 舒适型 ２０１０款 最新报价,经销商 电话 试驾／订车８仕嗍√焖市秦城区岷山路２０号６．５９万Ｎ浜菏薪岸区科技馆路特１号６...
6,汽车,戈蓝 ２．４ ＡＴ 旗舰升级版 ２０１２款 最新报价,经销商 电话 试驾／订车９愣省东莞市南城区莞太大道白马路段（周溪ＢＰ油站侧）＃矗埃埃福１１２...
7,汽车,华泰 宝利格,目前很多自主品牌的崛起都是从模仿世界老牌名车开始，这并不是一件坏事，这是一种让中国汽车业发展...
8,汽车,酷似卡宴 华泰新ＳＵＶ宝利格广州车展上市,华泰在推出自主轿车Ｂ１１后，又一款自主ＳＵＶ宝利格已经确定将在１１月下旬的广州车展上市正式上...
9,汽车,最受欢迎ＴＯＰ１０车型出炉 帝豪ＥＣ７脱颖而出,爱美之心人皆有之，汽车的外观炫感度所形成的冲击力，往往会给第一眼与之接触的消费者留下挥之不去...


In [3]:
data.shape

(5000, 3)

In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 3 columns):
类别      5000 non-null object
新闻标题    5000 non-null object
新闻内容    5000 non-null object
dtypes: object(3)
memory usage: 117.3+ KB


### 把数据集分为训练集和测试集

sk-learn库中train_test_split函数可以把数据集随机划分为训练集和测试集，训练集用于模型训练，测试集用于检验模型的优劣，random_state参数是指定随机种子。

In [5]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(data['新闻内容'], data['类别'], random_state=1)

### 中文分词

我们使用jieba库进行分词,并以空格把分词连成字符串。

In [6]:
def fenci(train_data):
    words_df = train_data.apply(lambda x:' '.join(jieba.cut(x)))
    return words_df

x_train_fenci = fenci(x_train)
x_train_fenci[:5]

Building prefix dict from the default dictionary ...
Dumping model to file cache C:\Users\ADMINI~1\AppData\Local\Temp\jieba.cache
Loading model cost 1.033 seconds.
Prefix dict has been built succesfully.


3186    中新网 上海 ６ 月 １ ４ 日电 （ 记者 于俊 ） 今年 的 父亲节 ， 人们 可 通过...
632     目前 正值 中 报 发布 之际 ， 多家 券商 认为 ， 上半年 银行 整体 利润 增速 下...
577     作为 中非 合作 论坛 的 重要 分 论坛 之一 ， 中 非金融 合作 论坛 将 于 １ ３...
2406    雅虎 体育讯 　 北京 时间 ７ 月 ３ 日 下午 ， 炎炎夏日 中 山东 球迷 终于 迎来...
4686    欧莱雅集团 的 药妆 品牌 理肤泉 （ 右图 ） 和 薇 姿 （ 左图 ） 。 　 资料 图...
Name: 新闻内容, dtype: object

In [7]:
x_train_fenci.head()

3186    中新网 上海 ６ 月 １ ４ 日电 （ 记者 于俊 ） 今年 的 父亲节 ， 人们 可 通过...
632     目前 正值 中 报 发布 之际 ， 多家 券商 认为 ， 上半年 银行 整体 利润 增速 下...
577     作为 中非 合作 论坛 的 重要 分 论坛 之一 ， 中 非金融 合作 论坛 将 于 １ ３...
2406    雅虎 体育讯 　 北京 时间 ７ 月 ３ 日 下午 ， 炎炎夏日 中 山东 球迷 终于 迎来...
4686    欧莱雅集团 的 药妆 品牌 理肤泉 （ 右图 ） 和 薇 姿 （ 左图 ） 。 　 资料 图...
Name: 新闻内容, dtype: object

### 引入停用词

In [8]:
infile = open("./dataset/stopwords.txt",encoding='utf-8')
stopwords_lst = infile.readlines()
stopwords = [x.strip() for x in stopwords_lst]

### 文本特征提取（词库表示法）
CountVectorizer旨在通过计数来将一个文档转换为向量。当不存在先验字典时，Countvectorizer作为Estimator提取词汇进行训练，并生成一个CountVectorizerModel用于存储相应的词汇向量空间。该模型产生文档关于词语的稀疏表示。下面举一个例子示范：

In [9]:
#用于转词向量的语料
yuliao = ['dog cat fish dog dog dog','cat eat fish','i like eat fish']

#sklearn库CountVectorizer转词向量
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer()
vector = cv.fit_transform(yuliao)
vector.todense()

matrix([[1, 4, 0, 1, 0],
        [1, 0, 1, 1, 0],
        [0, 0, 1, 1, 1]], dtype=int64)

In [10]:
#提取到的文本特征
cv.vocabulary_

{'dog': 1, 'cat': 0, 'fish': 3, 'eat': 2, 'like': 4}

从上面的例子可以看出，语料中每个词作为一个特征，词频数作为特征的值，如第一句中dog出现了4次，因此特征值为4。下面我们使用CountVectorizer把分词后的新闻文本转为向量。sklearn库中可以指定stopwords，我们把之前准备好的停用词表穿进去就好，这样我们的文本特征提取就做好啦。

In [11]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(stop_words=stopwords, max_features=5000)
cv.fit(x_train_fenci)

F:\anaconda\lib\site-packages\sklearn\feature_extraction\text.py:301: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['ain', 'aren', 'couldn', 'didn', 'doesn', 'don', 'hadn', 'hasn', 'haven', 'isn', 'lex', 'll', 'mon', 'null', 'shouldn', 've', 'wasn', 'weren', 'won', 'wouldn', '１２', 'ｌｉ', 'ｚｘｆｉｔｌ'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=5000, min_df=1,
        ngram_range=(1, 1), preprocessor=None,
        stop_words=['!', '"', '#', '$', '%', '&', "'", '(', ')', '*', '+', ',', '-', '--', '.', '..', '...', '......', '...................', './', '.一', '记者', '数', '年', '月', '日', '时', '分', '秒', '/', '//', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', '://', '::', ';', '<', '=', '>', '>>', '?', '@'...3', '94', '95', '96', '97', '98', '99', '100', '01', '02', '03', '04', '05', '06', '07', '08', '09'],
        strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
        tokenizer=None, vocabulary=None)

### 机器学习建模

这里我们使用朴素贝叶斯分类器，关于朴素贝叶斯算法，刘建平的博客写得非常不错，我就不再花时间整理啦，给大家推荐一波https://www.cnblogs.com/pinard/p/6069267.html

In [12]:
from sklearn.naive_bayes import MultinomialNB
classifier = MultinomialNB()
classifier.fit(cv.transform(x_train_fenci), y_train)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [13]:
classifier.score(cv.transform(fenci(x_test)), y_test)

0.8048

预测的准确率是80.5%，棒棒哒
### 文本特征提取(TF-IDF)

**TF-IDF**（term frequency–inverse document frequency），词频-逆文件频率。
>是一种用于资讯检索与资讯探勘的常用加权技术。TF-IDF是一种统计方法，用以评估一字词对于一个文件集或一个语料库中的其中一份文件的重要程度。字词的重要性随着它在文件中出现的次数成正比增加，但同时会随着它在语料库中出现的频率成反比下降

一个词语在一篇文章中出现次数越多, 同时在所有文档中出现次数越少, 越能够代表该文章.

下面直接给出一个词x的IDF的基本公式如下：

$IDF(x)= \lg {\frac{N}{N(x)}}$ 其中，N代表语料库中文本的总数，而N(x)代表语料库中包含词x的文本总数

$TF−IDF(x)=TF(x)∗IDF(x)$ 其中TF(x)指词x在当前文本中的词频

**词库表示法的缺点**：一些普遍出现的词，词频较高，看起来似乎是更重要的特征，但因为这个词普遍出现，这个词可能不是非常的重要。如果我们的向量化特征仅仅用词频表示就无法反应这一点。因此我们需要进一步的预处理来反应文本的这个特征，而这个预处理就是TF-IDF。

**用scikit-learn进行TF-IDF预处理**

In [14]:
from sklearn.feature_extraction.text import TfidfVectorizer
tv = TfidfVectorizer(stop_words=stopwords, max_features=5000)
tv.fit(x_train_fenci)
classifier.fit(tv.transform(fenci(x_train)), y_train)
classifier.score(tv.transform(fenci(x_test)), y_test)

F:\anaconda\lib\site-packages\sklearn\feature_extraction\text.py:301: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['ain', 'aren', 'couldn', 'didn', 'doesn', 'don', 'hadn', 'hasn', 'haven', 'isn', 'lex', 'll', 'mon', 'null', 'shouldn', 've', 'wasn', 'weren', 'won', 'wouldn', '１２', 'ｌｉ', 'ｚｘｆｉｔｌ'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


0.816

可以看出使用TF-IDF预处理后，得到的预测准确率有了明显的提高。


### N-gram模型

在朴素贝叶斯算法中，为了避免维度灾难，有一个大胆的假设，即X的n个维度之间相互独立:
$P(X_1=x_1,X_2=x_2,...X_n=x_n|Y=C_k)=P(X_1=x_1|Y=C_k)P(X_2=x_2|Y=C_k)...P(X_n=x_n|Y=C_k)$
这个假设下，条件分布大大的简化了，但是这也可能带来预测的不准确性。n个维度相互独立，就是完全没有考虑上下文语境，把每个词拆开单独看，这么看的话，猫吃鱼、鱼吃猫得到的特征向量是完全一样的。

N-gram模型就是假设$X_n$与附近n个词是相关的，比如当n=(1,2)时，猫吃鱼提取到的特征就是['猫','吃','鱼','猫吃','吃鱼']，为了平衡计算量和上下文关系，N一般取2或者3。朴素贝叶斯n个维度之间完全相互独立的假设，就是N-gram的n=1时的情况

TF-IDF是非常常用的文本挖掘预处理基本步骤，使用了IF-IDF并标准化以后，我们就可以使用各个文本的词特征向量作为文本的特征，进行分类或者聚类分析。

In [15]:
tv_2gram = TfidfVectorizer(stop_words=stopwords, max_features=5000, ngram_range=(1,2),lowercase = False)
tv_2gram.fit(x_train_fenci)
clf_2gram = MultinomialNB()
clf_2gram.fit(tv_2gram.transform(fenci(x_train)), y_train)
clf_2gram.score(tv_2gram.transform(fenci(x_test)), y_test)

F:\anaconda\lib\site-packages\sklearn\feature_extraction\text.py:301: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['ain', 'aren', 'couldn', 'didn', 'doesn', 'don', 'hadn', 'hasn', 'haven', 'isn', 'll', 'mon', 'shouldn', 've', 'wasn', 'weren', 'won', 'wouldn', '１２'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


0.8168

可以看出，指定N-gram模型n=2时，增加了附近2个词的关联，预测的准确率又上升了一丢丢，继续加油~